## Download data and install libraries

In [1]:
import os
!mkdir '/root/.kaggle'
os.chdir('/root/.kaggle')
![ -e kaggle.json ] && rm kaggle.json
!echo "{\"username\":\"honorisverum\",\"key\":\"1b763b7e5cdd751f5a7d53eaf02dd887\"}" >> kaggle.json
!chmod 600 ./kaggle.json
os.chdir('/content/')

In [2]:
!pip install kaggle --upgrade

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.9)


In [3]:
!kaggle competitions download -c kkbox-music-recommendation-challenge

 79% 33.0M/41.9M [00:00<00:00, 34.8MB/s]
100% 41.9M/41.9M [00:00<00:00, 61.3MB/s]
 98% 97.0M/98.8M [00:00<00:00, 83.6MB/s]
100% 98.8M/98.8M [00:00<00:00, 111MB/s] 
 80% 81.0M/101M [00:01<00:00, 75.8MB/s]
100% 101M/101M [00:01<00:00, 86.9MB/s] 
  0% 0.00/453k [00:00<?, ?B/s]
100% 453k/453k [00:00<00:00, 149MB/s]
  0% 0.00/1.29M [00:00<?, ?B/s]
100% 1.29M/1.29M [00:00<00:00, 186MB/s]
 96% 97.0M/101M [00:03<00:00, 45.3MB/s]
100% 101M/101M [00:03<00:00, 33.0MB/s] 


In [ ]:
!7z x members.csv.7z
!7z x sample_submission.csv.7z
!7z x song_extra_info.csv.7z
!7z x songs.csv.7z
!7z x test.csv.7z
!7z x train.csv.7z

In [5]:
!rm *.7z

In [ ]:
!pip install catboost

In [67]:
import copy
from collections import defaultdict
import random
import catboost
import xgboost
from itertools import combinations
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

## About data

In [3]:
pd.read_csv('./train.csv')[:5]

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [4]:
pd.read_csv('./songs.csv')[:5]

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [5]:
pd.read_csv('./members.csv')[:5]

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


## Data preparation

In [6]:
train_data = pd.read_csv('./train.csv')
user_data = pd.read_csv('./members.csv', parse_dates=['expiration_date', 'registration_init_time'])
item_data = pd.read_csv('./songs.csv')

train_data = train_data.merge(item_data, on='song_id', how='left')
train_data = train_data.merge(user_data, on='msno', how='left')

In [8]:
label_encoder = sklearn.preprocessing.LabelEncoder()

for col in ['msno', 'song_id', 'source_screen_name', 
            'source_system_tab', 'source_type', 'genre_ids', 
            'artist_name', 'composer', 'lyricist', 'gender']:
    train_data[col] = label_encoder.fit_transform(train_data[col].fillna('nan'))
    
for col in ['language', 'city', 'registered_via']:
    train_data[col] = label_encoder.fit_transform(train_data[col].fillna(-1))

train_data['time'] = train_data.index / len(train_data)

In [9]:
train_data[:5]

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date,time
0,8158,74679,1,7,7,1,206471.0,285,3277,14581,22882,9,0,0,2,2,2012-01-02,2017-10-05,0.000000e+00
1,17259,223479,3,8,4,1,284584.0,90,31960,64876,22882,9,11,24,0,3,2011-05-25,2017-09-11,1.355488e-07
2,17259,120758,3,8,4,1,225396.0,90,21372,45057,22882,9,11,24,0,3,2011-05-25,2017-09-11,2.710976e-07
3,17259,23707,3,8,4,1,255512.0,6,27439,36700,22882,1,11,24,0,3,2011-05-25,2017-09-11,4.066463e-07
4,8158,33308,1,7,7,1,187802.0,2,4472,8485,22882,9,0,0,2,2,2012-01-02,2017-10-05,5.421951e-07


In [10]:
len(train_data)

7377418

## Feature engineering

In [73]:
def past_count(df, cols):
    feature, cnt = [], defaultdict(lambda: 0)
    for v in df[col].values:
        cnt[v] += 0 if v not in cnt else 1
        feature.append(cnt[v])
    return feature

def value_count(df, col):
    vcnt_map = train_data[col].value_counts()
    feature = df[col].map(vcnt_map).fillna(0)
    return feature


def future_count(df, col):
    feature, df_reverse, cnt = \
        [], df.sort_index(ascending=False), defaultdict(lambda: 0)
    for v in df_reverse[col].values:
        feature.append(cnt[v] if v in cnt else 0)
        if v in cnt: cnt[v] += 1 
        else: cnt[v] = 1
    feature.reverse()
    return feature


def diff_count(df, col):
    last_time = df.groupby(df[col]).time.last()
    feature = df[col].map(last_time) - df.time
    return feature


def nexthearing_count(df, col):
    feature, df_reverse, nexthearing = [], df.sort_index(ascending=False), {}
    for v, t in zip(df_reverse[col], df_reverse.time):
        feature.append(t - nexthearing[v] if v in nexthearing else -1)
        nexthearing[v] = t
    feature.reverse()
    return feature

In [74]:
def make_features(df):

    categorical_columns = [
        'artist_name', 'city', 'composer', 'gender', 'genre_ids', 'language',
        'lyricist', 'msno', 'registered_via', 'song_id', 'source_screen_name',
        'source_system_tab', 'source_type', 'time',
    ]

    y = df.target
    X = pd.DataFrame()

    for col in categorical_columns:
        for func in [past_count, value_count, count_from_future,
                     diff_count, nexthearing_count]:
            X[f"{col}_{func.__name__}"] = func(df, col)

    for col in ['expiration_date', 'registration_init_time']:
        X[col] = df[col].apply(lambda x: x.toordinal())
    
    for col in ['song_length', 'bd']:
        X[col] = df[col]
    
    X['part_song_listened'] = df['song_length'] / X['msno_nexthearing_count']

    return X, y

In [75]:
# check on small dev subset

develop = train_data[:1000]
dev_X, dev_y = make_features(develop)

In [76]:
dev_X[:5]

,artist_name_past_count,artist_name_value_count,artist_name_count_from_future,artist_name_diff_count,artist_name_nexthearing_count,city_past_count,city_value_count,city_count_from_future,city_diff_count,city_nexthearing_count,composer_past_count,composer_value_count,composer_count_from_future,composer_diff_count,composer_nexthearing_count,gender_past_count,gender_value_count,gender_count_from_future,gender_diff_count,gender_nexthearing_count,genre_ids_past_count,genre_ids_value_count,genre_ids_count_from_future,genre_ids_diff_count,genre_ids_nexthearing_count,language_past_count,language_value_count,language_count_from_future,language_diff_count,language_nexthearing_count,lyricist_past_count,lyricist_value_count,lyricist_count_from_future,lyricist_diff_count,lyricist_nexthearing_count,msno_past_count,msno_value_count,msno_count_from_future,msno_diff_count,msno_nexthearing_count,registered_via_past_count,registered_via_value_count,registered_via_count_from_future,registered_via_diff_count,registered_via_nexthearing_count,song_id_past_count,song_id_value_count,song_id_count_from_future,song_id_diff_count,song_id_nexthearing_count,source_screen_name_past_count,source_screen_name_value_count,source_screen_name_count_from_future,source_screen_name_diff_count,source_screen_name_nexthearing_count,source_system_tab_past_count,source_system_tab_value_count,source_system_tab_count_from_future,source_system_tab_diff_count,source_system_tab_nexthearing_count,source_type_past_count,source_type_value_count,source_type_count_from_future,source_type_diff_count,source_type_nexthearing_count,time_past_count,time_value_count,time_count_from_future,time_diff_count,time_nexthearing_count,expiration_date,registration_init_time,song_length,bd,part_song_listened
0,1,1140,2,0.000017,-0.000016,1,2621293,349,0.000135,-5.421951e-07,1,499,0,0.000000,-1.000000,1,2961479,380,0.000135,-5.421951e-07,1,109513,17,0.000134,-1.640140e-05,1,1864789,216,0.000134,-1.355488e-07,1,3178798,392,0.000135,-1.355488e-07,1,5511,3,9.488415e-07,-5.421951e-07,1,2551743,377,0.000135,-5.421951e-07,1,215,0,0.0,-1.0,1,72342,8,0.000120,-5.421951e-07,1,167949,11,0.000124,-5.421951e-07,1,1967924,123,0.000134,-5.421951e-07,1,1,0,0.0,-1,736607,734504,206471.0,0,-3.808057e+11
1,1,303616,40,0.000133,-0.000002,1,1144863,258,0.000134,-1.355488e-07,1,1675706,186,0.000134,-0.000003,1,2118316,387,0.000134,-1.355488e-07,1,181713,17,0.000099,-1.355488e-07,1,1864789,215,0.000134,-1.355488e-07,1,3178798,391,0.000134,-1.355488e-07,1,622,3,6.777439e-07,-1.355488e-07,1,2814535,335,0.000135,-1.355488e-07,1,1,0,0.0,-1.0,1,3228202,729,0.000135,-1.355488e-07,1,3684730,776,0.000135,-1.355488e-07,1,1079503,269,0.000134,-1.355488e-07,1,1,0,0.0,-1,736583,734282,284584.0,24,-2.099495e+12
2,1,289,0,0.000000,-1.000000,1,1144863,257,0.000134,-1.355488e-07,1,4,0,0.000000,-1.000000,1,2118316,386,0.000134,-1.355488e-07,1,181713,16,0.000099,-4.066463e-07,1,1864789,214,0.000134,-2.710976e-07,1,3178798,390,0.000134,-1.355488e-07,1,622,2,5.421951e-07,-1.355488e-07,1,2814535,334,0.000135,-1.355488e-07,1,4,0,0.0,-1.0,1,3228202,728,0.000135,-1.355488e-07,1,3684730,775,0.000135,-1.355488e-07,1,1079503,268,0.000134,-1.355488e-07,1,1,0,0.0,-1,736583,734282,225396.0,24,-1.662841e+12
3,1,1,0,0.000000,-1.000000,1,1144863,256,0.000134,-4.066463e-07,1,1,0,0.000000,-1.000000,1,2118316,385,0.000134,-4.066463e-07,1,39,0,0.000000,-1.000000e+00,1,308752,38,0.000134,-4.473110e-06,1,3178798,389,0.000134,-1.355488e-07,1,622,1,4.066463e-07,-4.066463e-07,1,2814535,333,0.000135,-4.066463e-07,1,1,0,0.0,-1.0,1,3228202,727,0.000135,-4.066463e-07,1,3684730,774,0.000135,-4.066463e-07,1,1079503,267,0.000134,-4.066463e-07,1,1,0,0.0,-1,736583,734282,255512.0,24,-6.283396e+11
4,1,427,0,0.000000,-1.000000,1,2621293,348,0.000135,-1.355488e-07,1,412,0,0.000000,-1.000000,1,2961479,379,0.000135,-1.355488e-07,1,33496,3,0.000077,-4.066463e-07,1,1864789,213,0.000134,-1.355488e-07,1,3178798,388,0.000134,-1.355488e-07,1,5511,2,4.066463e-07,-1.355488e-07,1,25

In [77]:
dev_y[:5]

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

In [78]:
len(dev_X.columns)

75

In [42]:
train_shuffled = copy.deepcopy(train)
random.shuffle(train_shuffled)
X, y = make_features(train_shuffled)
len(X), len(y)

(7377418, 7377418)

In [40]:
del X
del y

## Train / test (on 5 kfolds)

In [ ]:
metrics = {'Logloss': 0.0, 'AUC': 0.0}
kf = KFold(n_splits=5)

for fold, (train_idxs, test_idxs) in enumerate(kf.split(X)):
    X_train, y_train = X.iloc[train_idxs], y.iloc[train_idxs]
    X_test, y_test = X.iloc[test_idxs], y.iloc[test_idxs]

    model = catboost.CatBoostClassifier(
        iterations=200, 
        learning_rate=0.03, 
        depth=7, 
        loss_function='Logloss',
        thread_count=50,
        random_seed=1,
        task_type="GPU",
        devices='0',
        verbose=False,
    )
    model.fit(X_train, y_train)

    pool = catboost.Pool(X_test, y_test)
    mt = model.eval_metrics(pool, ['Logloss', 'AUC'], thread_count=50)
    for k in mt:
        metrics[k] += mt[k][-1]
    
    del X_train
    del X_test

for k in metrics:
    metrics[k] /= 5

In [69]:
metrics

{'AUC': 0.7163594669879348, 'Logloss': 0.6038456790481755}

In [ ]:
#!kaggle competitions submit -c kkbox-music-recommendation-challenge -f 'submission.csv' -m '1st_try'